<a href="https://colab.research.google.com/github/aydawudu/Transformers_Practice/blob/main/Fine_Tunining_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 26.8 MB/s 
     |████████████████████████████████| 441 kB 71.0 MB/s 
     |████████████████████████████████| 163 kB 71.6 MB/s 
     |████████████████████████████████| 7.6 MB 70.5 MB/s 
     |████████████████████████████████| 115 kB 39.7 MB/s 
     |████████████████████████████████| 95 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 70.2 MB/s 
     |████████████████████████████████| 127 kB 77.6 MB/s 
     |████████████████████████████████| 115 kB 75.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [2]:
#get the data
from datasets import load_dataset
import numpy as np

In [3]:
raw_datasets=load_dataset("glue", "sst2")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [5]:
#select one of the dataset
raw_datasets['train']

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})

In [6]:
#let's see the attributes and methods the object has
dir(raw_datasets['train'])

['_TF_DATASET_REFS',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_local_temp_path',
 '_check_index_is_initialized',
 '_data',
 '_fingerprint',
 '_format_columns',
 '_format_kwargs',
 '_format_type',
 '_get_cache_file_path',
 '_get_output_signature',
 '_getitem',
 '_indexes',
 '_indices',
 '_info',
 '_iter',
 '_iter_batches',
 '_map_single',
 '_new_dataset_with_indices',
 '_output_all_columns',
 '_push_parquet_shards_to_hub',
 '_select_contiguous',
 '_select_with_indices_mapping',
 '_split',
 'add_column',
 'add_elasticsearch_index',
 'add_faiss_index',
 'add_faiss_index_from_exte

In [7]:
#lets check the type of object the dataset is
type(raw_datasets['train'])

datasets.arrow_dataset.Dataset

In [8]:
#let check the data attributes
raw_datasets['train'].data

MemoryMappedTable
sentence: string
label: int64
idx: int32
----
sentence: [["hide new secretions from the parental units ","contains no wit , only labored gags ","that loves its characters and communicates something rather beautiful about human nature ","remains utterly satisfied to remain the same throughout ","on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ","that 's far too tragic to merit such superficial treatment ","demonstrates that the director of such hollywood blockbusters as patriot games can still turn out a small , personal film with an emotional wallop . ","of saucy ","a depressed fifteen-year-old 's suicidal poetry ","are more deeply thought through than in most ` right-thinking ' films ",...,"sealed with a kiss ","very ugly ","why this project was undertaken ","the mill sci-fi film with a flimsy ending and ","halfhearted ","dilutes the pleasure of watching them ","the emotion is impressively true for being so hot-blooded , and ","can get your m

In [9]:
#let index the data object and select one
raw_datasets['train'][0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0}

In [10]:
#let select a subset
raw_datasets['train'][50000:50003]

{'sentence': ['glow ',
  'a classical dramatic animated feature ',
  'best espionage picture '],
 'label': [1, 1, 1],
 'idx': [50000, 50001, 50002]}

In [11]:
#lets check out its features
raw_datasets['train'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [12]:
from transformers import AutoTokenizer

In [13]:
# lets load the pretrained tokenizer using checkpoint="distilbert-base-uncased", since it faster than bert 
checkpoint="distilbert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [14]:
#let use the pretrained tokenizer
tokenized_sentences=tokenizer(raw_datasets['train'][0:3]['sentence'])
from pprint import pprint
pprint(tokenized_sentences)

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'input_ids': [[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102],
               [101,
                3397,
                2053,
                15966,
                1010,
                2069,
                4450,
                2098,
                18201,
                2015,
                102],
               [101,
                2008,
                7459,
                2049,
                3494,
                1998,
                10639,
                2015,
                2242,
                2738,
                3376,
                2055,
                2529,
                3267,
                102]]}


In [15]:
#let create a function for tokenization
def tokenize_fn(batch):
  return tokenizer(batch['sentence'], truncation=True)

In [16]:
#map the tokenize_fn to all the samples in our dataset
tokenized_datasets=raw_datasets.map(tokenize_fn, batched=True)

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [17]:
#import the traning argument class
from transformers import TrainingArguments


In [18]:
#define a training argument object and train on 1 epoch to avoid overfitting
training_args=TrainingArguments(
    'my_trainer',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=1,
)

In [19]:
#import the class for sequence classification and load up the model
from transformers.onnx.features import AutoModelForSequenceClassification
model=AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=2
)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

In [20]:
#lets check the type of our model object
type(model)

transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification

In [21]:
#let checkout the model architecture
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [22]:
#let use torch info to get the summary 
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
from torchinfo import summary
#summary(model, input_size=(16,512), dtypes=['torch.IntTensor'], device='cpu')
summary(model)

Layer (type:depth-idx)                                  Param #
DistilBertForSequenceClassification                     --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─LayerNorm: 3-3                              1,536
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             42,527,232
├─Linear: 1-2                                           590,592
├─Linear: 1-3                                           1,538
├─Dropout: 1-4                                          --
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0

In [24]:
#checks: save the model parameters before the traning process (to check if sll)
params_before=[]
for name, p in model.named_parameters():
  params_before.append(p.detach().cpu().numpy())


In [25]:
#import the trainer class
from transformers import Trainer

In [26]:
#import the loaded metrics function
from datasets import load_metric

In [27]:
#call the metric function, passing in our task
metric=load_metric("glue", "sst2")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


In [28]:
#lets check it out
metric.compute(predictions=[1,0,1], references=[1,0,0])


{'accuracy': 0.6666666666666666}

In [29]:
#let define a function to compute metrics
def compute_metrics(logits_and_labels):
  #metric =load_metric("glue", "sst2")
  logits, labels=logits_and_labels
  predictions=np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [30]:
trainer=Trainer(
    model, 
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)

In [31]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 67349
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8419
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encod

Epoch,Training Loss,Validation Loss
1,0.201700,0.367641


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 8
Saving model checkpoint to my_trainer/checkpoint-8419
Configuration saved in my_trainer/checkpoint-8419/config.json
Model weights saved in my_trainer/checkpoint-8419/pytorch_model.bin
tokenizer config file saved in my_trainer/checkpoint-8419/tokenizer_config.json
Special tokens file saved in my_trainer/checkpoint-8419/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=8419, training_loss=0.2671948120392676, metrics={'train_runtime': 437.936, 'train_samples_per_second': 153.787, 'train_steps_per_second': 19.224, 'total_flos': 518400815624736.0, 'train_loss': 0.2671948120392676, 'epoch': 1.0})

In [33]:
trainer.save_model("my_saved_model")

Saving model checkpoint to my_saved_model
Configuration saved in my_saved_model/config.json
Model weights saved in my_saved_model/pytorch_model.bin
tokenizer config file saved in my_saved_model/tokenizer_config.json
Special tokens file saved in my_saved_model/special_tokens_map.json


In [34]:
!ls

my_saved_model	my_svaed_model	my_trainer  sample_data


In [36]:
!ls my_saved_model

config.json	   special_tokens_map.json  tokenizer.json     vocab.txt
pytorch_model.bin  tokenizer_config.json    training_args.bin


now that we have a model let's demonstrate the use model

In [38]:
from transformers import pipeline

In [39]:
newmodel=pipeline('text-classification', model='my_saved_model', device=0)

loading configuration file my_saved_model/config.json
Model config DistilBertConfig {
  "_name_or_path": "my_saved_model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "vocab_size": 30522
}

loading configuration file my_saved_model/config.json
Model config DistilBertConfig {
  "_name_or_path": "my_saved_model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializ

In [40]:
newmodel('This movie is great!') #label 1 is positive class

[{'label': 'LABEL_1', 'score': 0.9993543028831482}]

In [44]:
newmodel('The movie is that bad') #label 0 is the negative class

[{'label': 'LABEL_0', 'score': 0.9982663989067078}]

In [42]:
newmodel('This food is not tasty')

[{'label': 'LABEL_0', 'score': 0.9966287016868591}]

Adding label names

In [45]:
!cat my_saved_model/config.json

{
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "vocab_size": 30522
}


In [46]:
import json

In [47]:
config_path='my_saved_model/config.json'
with open(config_path) as f:
  j=json.load(f)

j['id2label']={0:'negative', 1:'positive'}

with open(config_path, 'w') as f:
  json.dump(j, f, indent=2)

In [48]:
!cat my_saved_model/config.json

{
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "vocab_size": 30522,
  "id2label": {
    "0": "negative",
    "1": "positive"
  }
}

In [51]:
newmodel=pipeline('text-classification', model='my_saved_model', device=0)

loading configuration file my_saved_model/config.json
Model config DistilBertConfig {
  "_name_or_path": "my_saved_model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "negative",
    "1": "positive"
  },
  "initializer_range": 0.02,
  "label2id": null,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "vocab_size": 30522
}

loading configuration file my_saved_model/config.json
Model config DistilBertConfig {
  "_name_or_path": "my_saved_model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention

In [52]:
newmodel("This is a fun model")

[{'label': 'positive', 'score': 0.9993796348571777}]

let check whether all of our weights in our neural network was changed during training

In [55]:
model.named_parameters

<bound method Module.named_parameters of DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropou

In [53]:
params_after=[]
for name, p in model.named_parameters():
  params_after.append(p.detach().cpu().numpy())

In [57]:
for p1, p2 in zip(params_before, params_after):
  print(np.sum(np.abs(p1-p2)))

13394.419
92.34148
1.8242142
1.1396112
1306.4067
1.6635503
1289.829
0.0026388885
1191.6619
1.0899284
1119.3489
0.8492866
1.6720438
0.84606105
4927.3433
5.619396
4576.2124
0.8047422
1.5359199
0.70752835
1274.518
1.5034095
1256.2585
0.0026901565
1103.2852
0.8494714
1061.7048
0.7620292
1.4443743
0.74245846
4862.793
5.4133697
4432.8594
0.69172716
1.4150761
0.7633638
1252.096
1.4762015
1264.3054
0.0022822123
1094.9375
0.7816886
1072.6531
0.7207736
1.4617467
0.79246366
4878.613
5.499643
4297.6523
0.6626347
1.4333525
0.66405493
1286.7081
1.4791576
1300.9327
0.0030987258
1141.5825
0.74808776
1095.2819
0.69638264
1.4506185
0.713948
4793.5933
5.509891
4147.455
0.7597229
1.3476177
0.7365588
1182.7803
1.5263853
1175.8177
0.0014394608
950.2989
0.75132596
962.284
0.9325823
1.3377655
0.8949169
4360.443
4.947799
3414.71
0.752048
1.303602
0.7324358
1125.0948
1.3742514
1133.3186
0.0011871391
919.2952
0.8000469
910.6688
0.9443954
1.2900553
1.0973423
3507.2126
4.4841113
2994.2249
0.8607359
1.2554921
0.682

its shows that the weight were updatted during the traning process